In [1]:
import pandas as pd

In [2]:
file = pd.read_csv('events_up_to_01062018.csv',low_memory=False)

In [3]:
file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2341681 entries, 0 to 2341680
Data columns (total 23 columns):
timestamp                   object
event                       object
person                      object
url                         object
sku                         float64
model                       object
condition                   object
storage                     object
color                       object
skus                        object
search_term                 object
staticpage                  object
campaign_source             object
search_engine               object
channel                     object
new_vs_returning            object
city                        object
region                      object
country                     object
device_type                 object
screen_resolution           object
operating_system_version    object
browser_version             object
dtypes: float64(1), object(22)
memory usage: 410.9+ MB


Separar condition y storage segun si vio o compro <br>
Columnas de los modelos mas comprados y si la persona lo vio o no


In [6]:
conditions = file['condition'].dropna().unique()
storages = file['storage'].dropna().unique()
columns = []

for c in conditions:
    columns.append(c+'_compro')
    columns.append(c+'_vio')
for s in storages:
    columns.append(s+'_compro')
    columns.append(s+'_vio')
    
def viocompro(x):
    a = []
    df_c = x.loc[x.event=='conversion',['condition','storage']]
    df_v = x.loc[x.event=='viewed product',['condition','storage']]
    for c in conditions:
        a.append(c in list(df_c['condition']))
        a.append(c in list(df_v['condition']))
    for s in storages:
        a.append(s in list(df_c['storage']))
        a.append(s in list(df_v['storage']))
    return pd.Series(a)
viocompro = file.groupby('person').apply(viocompro)
viocompro.columns = columns


In [22]:
compras = file.loc[file.event=='conversion','model']
mascomprados = list(compras.value_counts().head(7).index)
def violosmascomprados (x):
    a = []
    b = x.loc[x.event=='viewed product','model'].values
    for c in mascomprados:
        a.append(c in b)
    return pd.Series(a)
mascomprados = file.groupby('person').apply(violosmascomprados)

In [23]:
mascomprados.columns = list(compras.value_counts().head(7).index)

In [24]:
mascomprados.describe()

,Samsung Galaxy J5,iPhone 5s,iPhone 6,iPhone 6S,Motorola Moto G2 3G Dual,Samsung Galaxy J7 Prime,Motorola Moto G4 Plus
count,38829,38829,38829,38829,38829,38829,38829
unique,2,2,2,2,2,2,2
top,False,False,False,False,False,False,False
freq,33861,29520,28721,30244,36728,34879,36433


## Exporto el nuevo archivo de features

In [29]:
features_1 = pd.read_csv('features_05.csv')

In [33]:
features_2 = pd.read_csv('features_test.csv')

In [46]:
repeated = ['busco_productos', 'clickeo_ad', 'hizo_checkout', 'entro_desde_motor',
       'hizo_conversion', 'filtro_busquedas', 'busco_marca', 'visito_sitio',
       'hizo_lead','primer_visita','ultima_visita','entro_direct','uso_computer']

In [53]:
features_06 = features_1.set_index('person').drop(repeated,axis=1).join(features_2.set_index('person'),lsuffix='_1')

In [54]:
features_06 = features_new.drop(['Unknown','Unknown.1'],axis=1)

In [56]:
features_07 = features_new.join(mascomprados).join(viocompro).drop(['Excelente', 'Muito Bom', 'Bom', 'Bom - Sem Touch ID', 'Novo',
       '32GB', '64GB', '128GB', '16GB', '8GB', '256GB', '4GB', '512MB'],axis=1)

In [59]:
features_07.to_csv('features_07.csv')